In [1]:
import sys
sys.path.append('../')

In [2]:
from textpair.single.paddle_bow import PaddleBowSim
from textpair.single.ann import Ann
from tqdm import tqdm

In [3]:
VOCAB_FILE = '../../paddle_models/PaddleNLP/similarity_net/data/term2id.dict'
MODEL_PATH = '../../paddle_models/PaddleNLP/similarity_net/model_files/simnet_bow_pairwise_pretrained_model/'

In [4]:
sim = PaddleBowSim(VOCAB_FILE, MODEL_PATH)

In [5]:
text1 = "人民团体是什么"
text2 = "人民团体是指"
ann1 = Ann(text1)
ann2 = Ann(text2)

In [6]:
res = sim(ann1, ann2)

Building prefix dict from the default dictionary ...
2019-06-25 18:11:51,946-DEBUG: Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
2019-06-25 18:11:51,949-DEBUG: Loading model from cache /tmp/jieba.cache
Loading model cost 0.880 seconds.
2019-06-25 18:11:52,828-DEBUG: Loading model cost 0.880 seconds.
Prefix dict has been built succesfully.
2019-06-25 18:11:52,829-DEBUG: Prefix dict has been built succesfully.


In [7]:
res

0.9515146315097809

In [8]:
for i in tqdm(range(10000)):
    ann1 = Ann(text1)
    ann2 = Ann(text2)
    res = sim(ann1, ann2)

100%|██████████| 10000/10000 [00:15<00:00, 638.37it/s]
